In [31]:
!pip install pandas
!pip install --upgrade instaloader


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import json
import time
import instaloader
import pandas as pd
from instaloader.exceptions import BadResponseException, ConnectionException, QueryReturnedBadRequestException

## Import Dataset

In [3]:
df = pd.read_excel('toDownloadjsons2.xlsx')
df

,username,followers,followings,likes,COMMENTS,description,date,num_tags,location,post_id,hashtags,num_hashtags,day_of_week,mention_count,total_points,hour
0,sport24,49785,177,2975,96,Ολυμπιακός και Εφές Αναντολού θα διεκδικήσουν ...,2017-04-30 00:28:29,0,NaN,BTe_60mh6_3,"#Devotion,#EuroLeague,#Olympiacos,#Spanoulis,#...",5,7,0,9197,1
1,portretexclusiveboutique,40814,1750,412,2,??,2017-04-19 22:05:25,0,NaN,BTE_mMkDlut,NaN,0,3,0,0,23
2,ancientmemes,31481,85,4386,17,#life_greece #greecetravelgr1_ #kings_greece #...,2017-04-30 00:23:00,-,-,BTe_Sp3hXF9,"#life_greece,#greecetravelgr1_,#kings_greece,#...",19,7,0,6808,1
3,ckourampis,12838,702,564,0,#Caldera view 3:2,2017-04-21 21:47:28,-,-,BTKHIfZlE_t,#Caldera,1,5,0,0,22
4,ckourampis,12838,702,359,0,Wo @alice_gao ・・・summer memories,2017-04-22 21:43:02,-,-,BTMra22lE1D,NaN,0,6,1,0,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,dina.nikolaou,12625,605,308,4,#oeufsdepaques #oeufs #lovemonvillage #dina_ni...,2017-04-15 21:59:29,-,-,BS6rvP4lxyi,"#oeufsdepaques,#oeufs,#lovemonvillage,#dina_ni...",10,6,0,19838,22
242,dina.nikolaou,12625,605,406,4,#paques #prentemps2017 #ανοιξη2017 #πασχα2017 ...,2017-04-15 21:49:36,-,-,BS6qm2SlAJt,"#paques,#prentemps2017,#ανοιξη2017,#πασχα2017,...",10,6,0,15838,22
243,dina.nikolaou,12625,605,280,0,#paques #cuisine_grecque #jukeros #chutneylove...,2017-04-16 16:56:43,-,-,BS8t4p0Fln_,"#paques,#cuisine_grecque,#jukeros,#chutneylove...",9,7,2,5919,17
244,interiorlifestylebyme,12323,681,325,35,God kväll finingar ? Hoppas ni har haft en här...,2017-04-19 22:01:36,-,-,BTE_KOJhqtt,NaN,0,3,0,0,23


## List processed post IDs

In [4]:
json_files = [f for f in os.listdir('posts') if f.endswith('.json')]
file_names = [os.path.splitext(f)[0] for f in json_files]

In [5]:
with open('invalid_posts.txt', 'r') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]

In [6]:
posts = file_names + lines

## Download post details

In [ ]:
import os
import json
import time
import instaloader

# Create an instance of Instaloader with custom user agent
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
loader = instaloader.Instaloader(user_agent=user_agent)

# Define login credentials
username = 'username'
password = 'password'

# Check if the session file exists, and if so, load the session
session_file = "session"
if os.path.exists(session_file):
    loader.load_session_from_file(username, filename=session_file)

# If the session is not loaded or invalid, log in with the provided credentials
if not loader.context.is_logged_in or not loader.test_login():
    try:
        loader.login(username, password)
        loader.save_session_to_file(filename=session_file)
    except instaloader.exceptions.BadCredentialsException:
        print("Invalid login credentials.")
        exit(1)
    except instaloader.exceptions.TwoFactorAuthRequiredException:
        print("Two-factor authentication is required. Please handle this case manually.")
        exit(1)
    except Exception as e:
        print("An error occurred during login:", e)
        exit(1)

# Continue with the rest of the script
START_INDEX = 1
END_INDEX = 25070

counter = 1
for post_id in df['post_id']:
    print(counter)
    counter += 1
    
    if counter < START_INDEX or counter > END_INDEX:
        continue
    
    if post_id in posts:
        continue
        
    try:
        # Get the post details using the post ID
        post = instaloader.Post.from_shortcode(loader.context, post_id)
        image_url = post.url
        tags = post.caption_hashtags
        mentions = post.caption_mentions
        description = post.caption
        date = post.date_local.strftime('%Y-%m-%d')
        likes = post.likes
        comments = post.comments
        timestamp = post.date_utc.timestamp()

        # Create a dictionary to store the post details
        post_dict = {
            "Image URL": image_url,
            "Tags": tags,
            "Mentions": mentions,
            "Description": description,
            "Date": date,
            "Likes": likes,
            "Comments": comments,
            "Timestamp": timestamp
        }

        # Write the post details dictionary to a JSON file
        file_name = os.path.join("posts", f"{post_id}.json")
        with open(file_name, "w") as json_file:
            json.dump(post_dict, json_file)
    except instaloader.exceptions.BadResponseException:
        print(post_id)
        with open('invalid_posts.txt', "a") as file:
            file.write('\n' + post_id)
    except instaloader.exceptions.ConnectionException:
        time.sleep(120)
        print("Connection exception occurred for post ID:", post_id)